In [1]:
import pandas as pd
import string
import nltk
import math
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import regex as re
import operator
import os
import sys
import time
import json
import csv
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)

In [16]:
dicoTfidf = {}
sys.path.append("C:/Users/Admin/Projet FeelCrypt/FeelCrypt/Valentin/")
import dataset_v
df = dataset_v.get_labeled_dataset(all_files = True)

Number of files loaded :  668


In [23]:
start_time = time.time()
dico = {}
dico = dict.fromkeys(df.date.unique(),0)
index = 0
# parcourir toutes les lignes du dataframe
for date in df.date.unique()[:50]:
    print(date)
    dicoDate = []
    #print(f'date = {date} | index {index}')
    # Si on est sur la même date que celle recupéré par la boucle for, on continue la tfidf actuelle
    while df["date"][index] == date and index < len(df)-1:
        # on rentre au moins une fois dedans
        dicoDate.append(df["text"][index])
        index = index + 1
    # On a récupéré tous les tweets de la date que l'on veut analyser avec tfidf
    # On le convertit en Series d'abord
    dicoDate = pd.Series(dicoDate)
    # Preprocessing des données
    dicoDate = preprocessing(dicoDate)
    # maintenant on peut appliquer tfidf
    uniqueWords = []
    for sentence in dicoDate:
        words = [word for word in sentence]
        for word in words:
            if word not in uniqueWords:
                uniqueWords.append(word)
    uniqueWords.sort()
    
    numOfWords = dict.fromkeys(uniqueWords, 0)
    for i in range(len(dicoDate)):
        try :
            for words in dicoDate[i]:
                numOfWords[words] += 1
        except KeyError:
            i += 1
    
    tf = computeTF(dicoDate, uniqueWords)
    idfs = dict.fromkeys(uniqueWords, 0)
    idfs = computeIDF(dicoDate, idfs, numOfWords)
    tfidf = computeTF_IDF(tf, idfs)
    
    columns = tfidf.columns.values
    dicoTFIDFWords = {}
    indexes = 0
    for label in columns:
        dicoTFIDFWords[label] = sum(tfidf[columns[indexes]])
        indexes = indexes + 1
    dicoTFIDFWords = sorted(dicoTFIDFWords.items(), key=operator.itemgetter(1), reverse=True)
    dicoMostImportantWords = dicoTFIDFWords[0:50]
    dicoWordsToKeep = []
    for l in dicoMostImportantWords:
        dicoWordsToKeep.append(l[0])
    #dico[date] = dicoMostImportantWords
    dico[date] = dicoWordsToKeep
print("Temps d execution tfidf : %s secondes ---" % (time.time() - start_time)) 

2015-12-15
2015-12-16
2015-12-17
2015-12-30
2015-12-31
2016-01-01
2016-01-18
2016-01-19
2016-01-20
2016-01-21
2016-01-25
2016-01-26
2016-01-27
2016-02-01
2016-02-08
2016-02-11
2016-02-12
2016-02-14
2016-02-25
2016-02-26
2016-02-27
2016-02-28
2016-02-29
2016-03-01
2016-03-02
2016-03-03
2016-03-04
2016-03-05
2016-03-06
2016-03-12
2016-03-13
2016-03-14
2016-03-15
2016-03-18
2016-05-02
2016-05-03
2016-05-04
2016-05-05
2016-05-06
2016-05-07
2016-05-08
2016-06-30
2016-07-01
2016-07-02
2016-07-03
2016-07-13
2016-07-26
2016-09-22
2016-11-14
2016-11-15
Temps d execution tfidf : 72.62757420539856 secondes ---


In [24]:
# Write to json file
with open('TFIDF.json', 'w') as fp:
    json.dump(dico, fp)

In [25]:
with open('TFIDF.json') as json_data:
    dataJson = json.load(json_data)
dataframeJson = json_normalize(dataJson)
csvFile = dataframeJson.to_csv('TFIDF.csv')

In [26]:
csv = pd.read_csv('TFIDF.csv')

In [27]:
csv

,Unnamed: 0,2015-12-15,2015-12-16,2015-12-17,2015-12-30,2015-12-31,2016-01-01,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-25,2016-01-26,2016-01-27,2016-02-01,2016-02-08,2016-02-11,2016-02-12,2016-02-14,2016-02-25,2016-02-26,2016-02-27,2016-02-28,2016-02-29,2016-03-01,2016-03-02,2016-03-03,2016-03-04,2016-03-05,2016-03-06,2016-03-12,2016-03-13,2016-03-14,2016-03-15,2016-03-18,2016-05-02,2016-05-03,2016-05-04,2016-05-05,2016-05-06,2016-05-07,2016-05-08,2016-06-30,2016-07-01,2016-07-02,2016-07-03,2016-07-13,2016-07-26,2016-09-22,2016-11-14,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-21,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-12-01,2016-12-02,2016-12-03,2016-12-05,2016-12-13,2016-12-14,2016-12-17,2016-12-20,2016-12-23,2016-12-24,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-07,2017-01-09,2017-01-10,2017-01-13,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-08,2017-02-09,2017-02-10,2017-02-11,2017-02-12,2017-02-13,2017-02-19,2017-02-21,2017-02-22,2017-02-27,2017-02-28,2017-03-02,2017-03-03,2017-03-04,2017-03-06,2017-03-07,2017-03-08,2017-03-12,2017-03-13,2017-03-14,2017-03-15,2017-03-16,2017-03-18,2017-03-19,2017-03-20,2017-03-21,2017-03-22,2017-03-23,2017-03-24,2017-03-27,2017-03-28,2017-03-29,2017-03-30,2017-04-08,2017-05-01,2017-05-02,2017-05-03,2017-05-04,2017-05-05,2017-05-06,2017-05-07,2017-05-08,2017-05-09,2017-05-10,2017-05-11,2017-05-12,2017-05-17,2017-05-18,2017-05-19,2017-05-20,2017-05-21,2017-05-22,2017-05-24,2017-05-26,2017-05-28,2017-05-29,2017-06-02,2017-06-03,2017-06-04,2017-06-05,2017-06-09,2017-06-10,2017-06-11,2017-06-12,2017-06-13,2017-06-14,2017-06-15,2017-06-18,2017-06-19,2017-06-22,2017-06-23,2017-06-24,2017-06-25,2017-06-26,2017-06-27,2017-06-28,2017-06-29,2017-07-16,2017-07-18,2017-07-22,2017-07-23,2017-07-24,2017-07-25,2017-07-26,2017-07-27,2017-07-28,2017-07-30,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10,2017-09-11,2017-09-12,2017-09-13,2017-09-14,2017-09-15,2017-09-16,2017-09-17,2017-09-18,2017-09-19,2017-09-21,2017-09-24,2017-09-25,2017-09-26,2017-09-27,2017-09-28,2017-09-29,2017-09-30,2017-10-01,2017-10-02,2017-10-03,2017-10-04,2017-10-05,2017-10-06,2017-10-07,2017-10-10,2017-10-11,2017-10-12,2017-10-13,2017-10-14,2017-10-15,2017-10-16,2017-10-17,2017-10-18,2017-10-19,2017-10-20,2017-10-21,2017-10-22,2017-10-23,2017-10-24,2017-10-25,2017-10-26,2017-10-27,2017-10-28,2017-10-29,2017-10-30,2017-10-31,2017-11-01,2017-11-02,2017-11-03,2017-11-04,2017-11-05,2017-11-06,2017-11-07,2017-11-08,2017-11-09,2017-11-10,2017-11-11,2017-11-12,2017-11-13,2017-11-14,2017-11-15,2017-11-16,2017-11-17,2017-11-18,2017-11-19,2017-11-20,2017-11-21,2017-11-22,2017-11-23,2017-11-24,2017-11-25,2017-11-26,2017-11-27,2017-11-28,2017-11-29,2017-11-30,2017-12-01,2017-12-02,2017-12-03,2017-12-04,2017-12-05,2017-12-06,2017-12-07,2017-12-08,2017-12-09,2017-12-10,2017-12-11,2017-12-12,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,2018-01-11,2018-01-12,2018-01-13,2018-01-14,2018-01-15,2018-01-16,2018-01-17,2018-01-18,2018-01-19,2018-01-20,2018-01-21,2018-01-22,2018-01-23,2018-01-24,2018-01-25,2018-01-26,2018-01-27,2018-01-28,2018-01-29,2018-01-30,2018-0

In [20]:
dico

{'2015-12-15': [('yes', 1.0986122886681098),
  ('mod', 0.5355044604617285),
  ('bullshit', 0.33882555199876413),
  ('btc', 0.2822931787105161),
  ('post', 0.2487015290152338),
  ('lol', 0.23104906018664842),
  ('disinformation', 0.2137953966012818),
  ('community', 0.20557461581454972),
  ('question', 0.1712401978033954),
  ('mods', 0.1612360690607877),
  ('bitcoin', 0.15333656903868106),
  ('btcdrak', 0.14447294195779428),
  ('users', 0.13924141335021611),
  ('rules', 0.12890105463044596),
  ('quite', 0.12614470030031782),
  ('ban', 0.12320745434240052),
  ('makes', 0.1162195863594582),
  ('put', 0.11539247572788099),
  ('one', 0.11270659344293393),
  ('seems', 0.1113567137376045),
  ('reddit', 0.11126780182947907),
  ('role', 0.10663802777845313),
  ('moderation', 0.1026962999046061),
  ('evidence', 0.09902102579427789),
  ('cant', 0.09618438028666743),
  ('comment', 0.0862582481127187),
  ('drama', 0.07954100206906575),
  ('score', 0.07336987670528688),
  ('ignoring', 0.072962861111

## Data Cleaning

In [3]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [4]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [5]:
def remove_links(text):
    words = [w for w in text if not len(w) > 20]
    return words

In [6]:
def stem_sentence(sentence):
    porter = PorterStemmer()
    for i in range(len(sentence)):
        word = porter.stem(sentence[i])
        sentence[i] = word
    return sentence

In [15]:
# Remove all "/u/", "/r", "/r", "\n" and numbers
def preprocessing(commentArray):
    for i in range(len(commentArray)):
        commentArray[i] = re.sub(r"(\/\w\/|\w\/)", "", commentArray[i])
        commentArray[i] = re.sub(r"\\n?t?", "", commentArray[i])
        commentArray[i] = re.sub(r"[0-9]*", "", commentArray[i])
    
    # punctuation
    commentArray = commentArray.apply(lambda x: remove_punctuation(x))
    tokenizer = RegexpTokenizer(r'\w+')
    commentArray = commentArray.apply(lambda x : tokenizer.tokenize(x.lower()))
    #file = file[file['body'].map(len) > 4]

    # remove small words
    for i in range(len(commentArray)):
        try :
            for word in commentArray[i]:
                if len(word) < 3:
                    commentArray[i].remove(word)
        except KeyError :
            continue
    
    # remove stopwords
    commentArray = commentArray.apply(lambda x : remove_stopwords(x))

    # reddit link
    commentArray = commentArray.apply(lambda x : remove_links(x))
    
    return commentArray

### Term Frequency (TF)

In [8]:
def computeTF(file, uniqueWords):
    tf = []
    index = 0
    
    try :
        for sentence in file:
            tf.append(dict.fromkeys(uniqueWords, 0))
            for word in sentence:
                tf[index][word] += 1 / len(sentence)
            index += 1
    except KeyError :
        print("error")
    return tf

### Inverse Data Frequency (IDF)

In [9]:
def computeIDF(file, idfs, numOfWords):
    # Pour chaque doc, pour chaque mot on regarde sa tf
    for x, y in idfs.items():
        count = 0
        for i in range(len(file)):
            try:
                for word in file[i]:
                    if x == word:
                        count += 1
                        break
            except KeyError:
                i += 1
        if count != 0:
            idfs[x] = math.log(numOfWords[x] / count)
    return idfs

### TF-IDF

In [10]:
def computeTF_IDF(tf, idfs):
    tab = []
    for i in range(len(tf)):
        dico = {}
        for mot in tf[i]:
            dico[mot] = tf[i][mot] * idfs[mot]
        tab.append(dico)
    tfidf = pd.DataFrame(tab)
    return tfidf